#Transfer Learning with RensorFlow Part 3 : Scalingup

(Food vision mini)

we've seen the power of transfer learning feature extraction and fine-tuning
, now it's time to scale up to all of the classes in Food101


In [ ]:
#using the helper funtions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
#Import series of helper function from the helper.py module
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [ ]:
## 101 Food Classes: working with less data trying to beat original Food101 paper with 10% of the training data.
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

In [ ]:
unzip_data("101_food_classes_10_percent.zip")
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [ ]:
#walk through the directories
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                  label_mode="categorical",
                                                                                  image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle = False)#don't shuffle test data for prediction analysis.


## Train a big dog model with transfer learning on 10% of 101 food classes

steps:

* Create a ModelCheckpoint callback
* Create a data augmentation layer to build data aumentation right into the model
* Build a headless (no top layers) funtional EfficientNetB0 backboned-model
* compile our model
* Feature extract for 5 full passes(5 epochs on the train dataset and validate on 15% of the test data, to save epoch time)


In [ ]:
#create checkpoint callback
checkpoint_path = "101_classes_10_percent_data_model_checkpoint.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [ ]:
#create data augmentation layer to incorporate it
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip,RandomRotation,RandomHeight,RandomWidth,RandomZoom

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomHeight(0.2),
    RandomWidth(0.2),
    RandomZoom(0.2),
    #Rescaling(1/255.) we need to rescale for image classification using resnet efficientnet automatically makes it normalized

],name="data_augmentation")

In [ ]:
#Setup the base model and freeze its layers (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

# Setup model architecture wth trainable top layers
inputs = layers.Input(shape=(224,224,3),name="input_layer")
x = data_augmentation(inputs)#Augment images (happens during training phase)
x = base_model(x,training=False)#base model in inference mode so weights which need to stay frozen , stay frozen)
x = layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),activation="softmax",name="output_layer")(x)
model = tf.keras.Model(inputs,outputs)


In [ ]:
print(outputs)

In [ ]:
#summary of the model created
model.summary()

In [ ]:
# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
#fit
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs = 5,# fit for 5 epochs basic
                                           validation_data = test_data,
                                           validation_steps = int(0.15 * len(test_data)),
                                           callbacks = [checkpoint_callback])

In [ ]:
#Evaluate on the whole test dataset
fine_tuneing_results = model.evaluate(test_data)
fine_tuneing_results

In [ ]:
#plot loss curves
plot_loss_curves(history_all_classes_10_percent)

#Fine-tuning

In [ ]:
#Unfreeze all of the layers in the base model
base_model.trainable = True

#Refreeze every layer except last 5
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [ ]:
for layer in base_model.layers:
  print(layer.name, layer.trainable)

In [ ]:
#Recompile model with lower learning rate (as it is better to lower learning rate after fine-tuning)
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=["accuracy"])

In [ ]:
#fine tune for 5 more epochs
fine_tune_epochs = 10 #model has already done 5 epochs (feature extraction), this is the total number of epochs including the previous training)
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                              epochs = fine_tune_epochs,
                              validation_data = test_data,
                              validation_steps = int(0.15 * len(test_data)),
                              initial_epoch = history_all_classes_10_percent.epoch[-1],

)

In [ ]:
#evaluate the model now
all_classes_10_percent_fine_tune = model.evaluate(test_data)
all_classes_10_percent_fine_tune

In [ ]:
#plotting loss curves
plot_loss_curves(history_all_classes_10_percent_fine_tune)

In [ ]:
from helper_functions import compare_historys

In [ ]:
#compare the histories
compare_historys(original_history = history_all_classes_10_percent,
                  new_history = history_all_classes_10_percent_fine_tune,
                  initial_epochs = 5)

#Saving and loading our model
To use the model in an external application, we'll need to save it and export it

In [ ]:
#save our fine-tuned model
model.save("drive/MyDrive/tensor_flow/101_food_classes_10_percent_data_model.keras")

In [ ]:
#load model
load_model = tf.keras.models.load_model("drive/MyDrive/tensor_flow/101_food_classes_10_percent_data_model.keras")

In [ ]:
#evaluate loaded model and compare performance to pre-saved model
loaded_model_results = load_model.evaluate(test_data)
loaded_model_results

In [ ]:
#The results from the model before it is saved

all_classes_10_percent_fine_tune


# New model

In [ ]:
import tensorflow as tf
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip


In [ ]:
#using the helper funtions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
import tensorflow as tf
from helper_functions import unzip_data

In [ ]:
unzip_data("/content/06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
## 101 Food Classes: working with less data trying to beat original Food101 paper with 10% of the training data.
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

In [ ]:
unzip_data("101_food_classes_10_percent.zip")

In [ ]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                  label_mode="categorical",
                                                                                  image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle = False)#don't shuffle test data for prediction analysis.


In [ ]:
import tensorflow as tf

# Load the old model using TFSMLayer (Inference-only)
base_model2 = tf.keras.layers.TFSMLayer(
    "/content/06_101_food_class_10_percent_saved_big_dog_model",  # Replace with your model path
    call_endpoint="serving_default"  # Replace with the correct endpoint if necessary
)




In [ ]:
# Define inputs
inputs = tf.keras.Input(shape=(224, 224, 3))  # Ensure this matches your image size

# Apply data augmentation (if needed)
x = data_augmentation(inputs)

# Pass through base model in inference mode
x = base_model2(x, training=False)  # Make sure base_model2 is in inference mode

# 🚀 Directly connect to Dense layer (REMOVE GlobalAveragePooling2D)
num_classes = len(train_data_all_10_percent.class_names)  # Ensure this variable is correct
# 🔥 Extract the correct tensor from the dictionary
x = x['dense_8']  # Replace 'dense_8' with the correct key if different
outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

# Define final model
model2 = tf.keras.Model(inputs, outputs)

In [ ]:


# Compile the model (Use categorical_crossentropy for one-hot encoded labels)
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=['accuracy']
)

# Print model summary
model2.summary()


In [ ]:
historyofimportedlibrary = model2.fit(train_data_all_10_percent,
                                     epochs =5,
                                     validation_data = test_data,
                                     validation_steps = int(0.15 * len(test_data))
                                     )



In [ ]:
results_downloaded_model = model.evaluate(test_data)
results_downloaded_model

In [ ]:
preds_probs = model2.predict(test_data,verbose =1 )

In [ ]:
len(preds_probs)

In [ ]:
preds_probs.shape

In [ ]:
#what does the first prediction probability array look like
preds_probs[0],len(preds_probs[0],sum(preds_probs[0]))

In [ ]:
#finding the probability of the first image.
print(f"The class with the highest probability is {preds_probs[0].argmax()} with a probability of {preds_probs[0].max()}")

In [ ]:

#Get the pred classes of each label
pred_classes = preds_probs.argmax(axis=1)

#How do the look?
pred_classes[:10]

len(pred_classes)

In [ ]:
#To ge our test labels we need to unravel our test_data BatchDataset

y_labels = []
for images, labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())
y_labels[:10] #look at tge first 10


#Evaluating our model's predictions
One way to check that our model's predictios array is in the same order as our test labels array is to find the accuracty score.


In [ ]:
results_downloaded_model

In [ ]:
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_true = y_labels,
                                  y_pred = pred_classes)
sklearn_accuracy

In [ ]:
import numpy as np
np.isclose(results_downloaded_model[1],sklearn_accuracy)

Confusion matrix

In [ ]:
#get a list of classes names
class_names = test_data.class_names
class_names

In [ ]:
make_confusion_matrix(y_true = y_labels,
                      y_pred = pred_classes,
                      classes = class_names,
                      figsize = (100,100),
                      text_size = 20,
                      savefig = True)

In [ ]:
## Let's evaluate for classification report
from sklearn.metrics import classification_report
print(classification_report(y_true = y_labels,
                            y_pred = pred_classes))

In [ ]:
#Creating better visualization
classification_report_dict = classification_report(y_true = y_labels,
                                                    y_pred = pred_classes,
                                                    output_dict = True)

classification_report_dict

In [ ]:
class_names[98]

In [ ]:
classification_report_dict["99"]["f1-score"]

In [ ]:
#create empty dictionar
class_f1_scores = {}
# loop through classification report dictionary items
for k,v in classification_report_dict.items():
  if k == "accuracy":
    continue
  class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
#trun f1-scores into dataframe for visualization
import pandas as pd
f1_scores_df = pd.DataFrame({"class_names":list(class_f1_scores.keys()),
                             "f1-score": list(class_f1_scores.values())}).sort_values("f1-score",ascending = False)
f1_scores_df

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (12,25))
scores = ax.barh(range(len(f1_scores_df)),f1_scores_df["f1-score"].values)
ax.set_yticks(range(len(f1_scores_df)))
ax.set_yticklabels(f1_scores_df["class_names"])
ax.set_xlabel("f1-score")
ax.set_title("f1-scores for 101 different food classes")
ax.invert_yaxis(); #to reverse the order of plot

#Visualizing predictions on custom images

* Read in a targe image filepath using tf.io.read_file()
* Turn the image into a Tensor using tf.io.decode_image()
* Resize the image into a Tensor to be the same size as the images our model has trained on using tf.image.resize()
* Scale the image to get all of the pixel values between 0 & 1 (if necessary)


In [ ]:
# creat a function to load and prepare images
def load_and_prep_image(filename, img_shape = 224, scale = True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224,224,3)

  args:
   filename(str): Path to target image
   image_shape (int): height/width dimension of target imagesize
   scale (bool): whether to scale pixel values to range(0,1) or not by default set to true

  Returns:
   Image tensor of image (image_shape, image_shape, 3)
  """
  #read the file
  img = tf.io.read_file(filename)
  #Decode image into tensor
  img = tf.image.decode_image(img,channels=3)
  #resize the image
  img = tf.image.resize(img, size = [img_shape,img_shape])

  #scale? yes or no
  if scale:
    #rescaling the image (if necessary)
    return img/255.
  else:
    return img# no need for efficient net models



1. Load a few random images from the test dataset
2. Make predictions on the loaded image
3. plot the original images along with the model's prediction

In [ ]:
#Make preds on a series of random images
import os
import random
plt.figure(figsize=(17,10))
for i in range(3):
  # choose a random image from a random class
  class_name = random.choice(class_names)
  filename = random.choice(os.listdir(test_dir + "/" + class_name))
  filepath = test_dir + "/" + class_name + "/" + filename

  #load the image and make predictions
  img = load_and_prep_image(filepath, scale = False)
  pred = model.predict(tf.expand_dims(img,axis=0)) # the data need to be same as the expected.
  pred_class = class_names[pred.argmax()]

  #plot the images
  plt.subplot(1,3,i+1)
  plt.imshow(img/255.)
  if class_name == pred_class:
    title_color = "g"
  else:
    title_color = "red"
  plt.title(f"Actual: {class_name} , Pred: {pred_class}, prob : {preds_probs}",color = title_color)
  plt.axis(False)

#Finding the most wrong predictions

view the wrong predictions with the highest prediction probability(or highest loss)

this will give insights such as:

* Data issues(wrong labels)
* Confusion classes

To find out where our model is most wrong we do the following:
1. Get all of the image file paths in the test dataset using list_files() method
2. Create a pandas Dataframe of the image filepaths, ground truth labels, predicted classes(from our model) , max prediction probabilities, prediction class names, ground truth class names.
3. use our DataFrame to find all the wrong predictions (where the ground truth label doen't match the prediction).

4. Sort the DataFrame based on wrong predictions (have the highest prediction probability predictions at the top).
5. visualize the images with the highest prediction probabilities but have the wrong prediction.

In [ ]:
# 1. Get all of the image file paths in the test dataset
filepaths = []
for filepath in test_data.list_files("/content/101_food_classes_10_percent/test/*/*.jpg",shuffle=False): # first * means every directory in test and second * means every file.jpg in the selected directory before.
   filepaths.append(filepath.numpy())
filepaths[:10]

In [ ]:
#2. Create a dataframe of different parameters for each of our test images
import pandas as pd
import pandas as pd

pred_df = pd.DataFrame({
    "img_path": filepaths,
    "y_true": y_labels,
    "y_pred": pred_classes,
    "pred_conf": preds_probs.max(axis=1),
    "y_true_classname": [class_names[i] for i in y_labels],
    "y_pred_classname": [class_names[i] for i in pred_classes]
})

# Display the first few rows
pred_df.head()


In [ ]:
# 3. find out our DataFrame which predictions are wrong
pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head()

In [ ]:
#4. sort our dataframe to have most wrong predictions at the top
top_100_wrong = pred_df[pred_df["pred_correct"]==False].sort_values("pred_conf",ascending = False)[:100]
top_100_wrong.head(20)#get the 20 samples of 100 dataframe

In [ ]:
# 5. visualize the test data samples which have the wrong prediction but highest pred probability.
images_to_view = 9
start_index = 0
plt.figure(figsize = (10,10))
for i,row in enumerate(top_100_wrong[start_index:start_index + images_to_view].itertuples()):
  plt.subplot(3,3,i+1)
  img = load_and_prep_image(row[1],scale=False)
  _,_,_,_,pred_prob,y_true_classname,y_pred_classname, _  row # only access the required columns
  plt.imshow(img/255.)
  plt.title(f"Actual: {y_true_classname}, Pred: {y_pred_classname},\n Prob: {pred_prob}")
  plt.axis(False)

#Testing the model

In [ ]:
#get custom images
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

In [ ]:
unzip_data("custom_food_images.zip")

In [ ]:
#Get the custom food images filepaths
custom_food_images = ["custom_food_image/"+img_path for img_path in os.listdir("custom_food_images")]
custom_food_images

In [ ]:
#make predictions on and plot custom food images
for img in custom_food_images:
  img = load_and_prep_image(img,scale = False)# don't need to scale for our EfficientNetB0
  pred = model.predict(tf.expand_dims(img,axis=0))# make prediction on image with shape [1,224,224,3] same as the model trained on
  pred_class = class_names[pred.argmax()]#get the index with the hightes prediction probability
  plt.figure()
  plt.imshow(img/255.)
  plt.title(f"pred: {pred_class}, prob: {pred.max(): .2f}")
  plt.axis(False)